In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import xgboost as xgb
import warnings
import yfinance as yf 
import pandas as pd

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


ticker="AAPL"
    
    #나스닥 파일 불러오기
data_path = 'data/nasdq_20241104.csv'
data = pd.read_csv(data_path)

    
start_date = '2010-01-04'
end_date = '2024-10-25'
stock_data = yf.download(ticker, start=start_date, end=end_date)
    
    #AAPL 데이터 전처리
stock_data.reset_index(inplace=True)
stock_data['Date'] = stock_data['Date'].dt.strftime('%Y-%m-%d')
stock_data.columns = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
stock_data.drop(columns=['Adj Close'], inplace=True)
    
    #Nasdaq 데이터 전처리
#global_data = data.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])

    #AAPL, Nasdaq merge
stock_merge_data = pd.merge(stock_data, data, on='Date', how='inner')

    #merge data 전처리
stock_merge_data['Date'] = pd.to_datetime(stock_merge_data['Date'])
stock_merge_data.set_index('Date', inplace=True)
stock_merge_data.columns = stock_merge_data.columns.str.replace(' ', '')

stock_merge_data['Close_InterestRate_Corr'] = stock_merge_data['Close'].rolling(252).corr(stock_merge_data['InterestRate'])
stock_merge_data['Close_VIX_Corr'] = stock_merge_data['Close'].rolling(252).corr(stock_merge_data['VIX'])
stock_merge_data['Rolling_Volatility'] = stock_merge_data['Close'].rolling(window=30).std()

stock_merge_data['Daily_Return'] = stock_merge_data['Close'].pct_change()
stock_merge_data['Volatility'] = stock_merge_data['Close'].rolling(window=30).std() # 이동 표준 편차를 계산하라
stock_merge_data['Rolling_Mean_Close'] = stock_merge_data['Close'].rolling(window=30).mean()
stock_merge_data.dropna(inplace=True)

stock_merge_data_head = stock_merge_data.iloc[:-10] # 앞쪽 
stock_merge_data_tail = stock_merge_data.iloc[-10:] # 뒷쪽 10개
    

X = stock_merge_data_head.drop(['Close', 'High', 'Low', 'Volume'], axis=1)  # Ensure 'Close' is dropped to create the feature set
y = stock_merge_data_head['Close']  # Target variable is 'Close' price



"""
    # Step 1: Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Step 2: Check for NaN values and handle them
    # Using forward fill to handle NaN values (you can adjust this as needed)
X.fillna(method='ffill', inplace=True)

    # Step 3: Standardize the data
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

    # Step 4: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # 마지막 테스트할꺼: Define target variable and features
    Final_X = stock_merge_data_tail.drop(['Close', 'High', 'Low', 'Volume'], axis=1)  # Ensure 'Close' is dropped to create the feature set
    Final_y = stock_merge_data_tail['Close']  # Target variable is 'Close' price
    Final_X.replace([np.inf, -np.inf], np.nan, inplace=True)
    Final_X.fillna(method='ffill', inplace=True)

    X_Final_scaled = scaler.transform(Final_X)

   # Initialize and train a random forest model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

# Make predictions
    y_pred_rf = rf.predict(X_test)
    y_pred_Final = rf.predict(X_Final_scaled)
    
    Final_data = stock_merge_data_tail[['Open','Close']]
    Final_data['Pred_Close'] = y_pred_Final
    Final_data['Close_diff'] = Final_data['Pred_Close'] - Final_data['Close']
    Final_data['diff_per'] = (Final_data['Close_diff'] / Final_data['Close']) * 100.0

    Final_predict = Final_data['Close_diff'][-1]
    if Final_predict>0:
        return 'UP',Final_predict
    else:
        return 'DOWN',Final_predict
"""


[*********************100%***********************]  1 of 1 completed


"\n    # Step 4: Split the data into training and testing sets\n    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)\n\n    # 마지막 테스트할꺼: Define target variable and features\n    Final_X = stock_merge_data_tail.drop(['Close', 'High', 'Low', 'Volume'], axis=1)  # Ensure 'Close' is dropped to create the feature set\n    Final_y = stock_merge_data_tail['Close']  # Target variable is 'Close' price\n    Final_X.replace([np.inf, -np.inf], np.nan, inplace=True)\n    Final_X.fillna(method='ffill', inplace=True)\n\n    X_Final_scaled = scaler.transform(Final_X)\n\n   # Initialize and train a random forest model\n    rf = RandomForestRegressor(n_estimators=100, random_state=42)\n    rf.fit(X_train, y_train)\n\n# Make predictions\n    y_pred_rf = rf.predict(X_test)\n    y_pred_Final = rf.predict(X_Final_scaled)\n    \n    Final_data = stock_merge_data_tail[['Open','Close']]\n    Final_data['Pred_Close'] = y_pred_Final\n    Final_data['Close_diff'] = 

In [19]:
X

,Open,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil,Close_InterestRate_Corr,Close_VIX_Corr,Rolling_Volatility,Daily_Return,Volatility,Rolling_Mean_Close
Date,,,,,,,,,,,,,,
2010-12-31,11.533929,0.18,1.3269,17.75,0.18,0.13,1421.099976,91.379997,0.642281,-0.182643,0.185331,-0.003399,0.185331,11.390191
2011-01-03,11.630000,0.17,1.3371,17.61,0.18,0.19,1422.599976,91.550003,0.633936,-0.190885,0.183933,0.021732,0.183933,11.415357
2011-01-04,11.872857,0.17,1.3306,17.38,0.16,0.18,1378.500000,89.379997,0.625355,-0.200087,0.177785,0.005219,0.177785,11.444595
2011-01-05,11.769643,0.17,1.3171,17.02,0.16,0.18,1373.400024,90.300003,0.615647,-0.210336,0.191963,0.008180,0.191963,11.469167
2011-01-06,11.954286,0.17,1.3027,17.40,0.15,0.17,1371.400024,88.379997,0.605810,-0.220130,0.190102,-0.000808,0.190102,11.498929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-04,227.899994,4.83,1.0961,19.21,0.09,4.83,2645.800049,74.379997,-0.431531,0.122000,3.916647,0.005007,3.916647,224.931667
2024-10-07,224.500000,4.83,1.0976,22.64,0.09,4.83,2644.800049,77.139999,-0.431909,0.137445,3.942895,-0.022531,3.942895,224.760001
2024-10-08,224.300003,4.83,1.0970,21.42,0.09,4.83,2615.000000,73.570000,-0.437076,0.150501,3.921399,0.018404,3.921399,224.713001
